In [1]:
# add parent directory to path: enable import from parent dir
import sys
sys.path.append('../')

from environment import SmartBrokerEnv
from agents.a2c import A2C
from networks.a2c.actor import Actor
from networks.a2c.critic import Critic
from networks.a2c_lstm.actor import ActorLSTM
from networks.a2c_lstm.critic import CriticLSTM

import gym
import matplotlib.pyplot as plt
import torch

In [2]:
norm_cols = []
cols = ['date'] + ['open', 'high', 'low', 'close', f'Volume XRP']
batch_dur = 30
env = SmartBrokerEnv(
    batch_dur=batch_dur,
    df_info={
        'start_date': '2021-02-01',
        'end_date': '2021-03-01',
        'norm_cols': norm_cols,
        'cols': cols,
    },
    portfolio={

    },
)

actor_model = ActorLSTM(
    state_dim=batch_dur*3+3,
    action_dim=3,
)
critic_model = Critic(state_dim=batch_dur*3+3)
a2c = A2C(
    env=env, 
    actor=actor_model,
    critic=critic_model,
    n_actns=3,
    actor_optmz=torch.optim.Adam(actor_model.parameters(), lr=0.00001),
    critic_optmz=torch.optim.Adam(critic_model.parameters(), lr=0.00001),
    hyprprms={
        'gamma': 0.995,
    },
    log_freq=1,
    p_net_type='lstm',
    c_net_type='nn',
    load_models=False,
    mdl_pth='../models/a2c_al_cn',
)

In [3]:
a2c.run(5000)


Ep: 0 | TS: 19970 | L: 2255111101.128 | R: -51.92 | P: 50.07 | R.Avg P: 50.07 | NW: 150.07 | R.Avg NW: 150.07 | R.U: 153
Ep: 1 | TS: 39940 | L: 2232754193.303 | R: -65.62 | P: 6.16 | R.Avg P: 28.12 | NW: 106.16 | R.Avg NW: 128.12 | R.U: 130
Ep: 2 | TS: 59910 | L: 2210529646.845 | R: -68.05 | P: -1.89 | R.Avg P: 18.11 | NW: 98.11 | R.Avg NW: 118.11 | R.U: 120
Ep: 3 | TS: 79880 | L: 2188446591.476 | R: -66.45 | P: 4.45 | R.Avg P: 14.7 | NW: 104.45 | R.Avg NW: 114.7 | R.U: 116
Ep: 4 | TS: 99850 | L: 2166509360.888 | R: -77.62 | P: -29.92 | R.Avg P: 5.77 | NW: 70.08 | R.Avg NW: 105.77 | R.U: 107
Ep: 5 | TS: 119820 | L: 2144718014.015 | R: -73.61 | P: -17.62 | R.Avg P: 1.88 | NW: 82.38 | R.Avg NW: 101.88 | R.U: 103
Ep: 6 | TS: 139790 | L: 2123079024.513 | R: -65.24 | P: 7.28 | R.Avg P: 2.65 | NW: 107.28 | R.Avg NW: 102.65 | R.U: 103
Ep: 7 | TS: 159760 | L: 2101591908.897 | R: -72.44 | P: -14.89 | R.Avg P: 0.46 | NW: 85.11 | R.Avg NW: 100.46 | R.U: 101
Ep: 8 | TS: 179730 | L: 2080257226.463

KeyboardInterrupt: 

In [4]:
a2c.evaluate(start_dt='2021-01-02 07:00', duration=30)

IndexError: index 0 is out of bounds for axis 0 with size 0

## Visualizations

### Training 

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(16, 12))
r_avg_rewards = []
r_avg_profits = []
r_avg_bal = []
r_avg_units_held = []
r_avg_loss = []
r_avg_net_worth = []
count = 0

for _, log in a2c.logs.items():
    r_avg_rewards.append(log['r_avg_reward'])
    r_avg_profits.append(log['r_avg_profit'])
    r_avg_bal.append(log['r_avg_bal'])
    r_avg_units_held.append(log['r_avg_units_held'])
    r_avg_loss.append(log['r_avg_loss'])
    r_avg_net_worth.append(log['r_avg_net_worth'])
    count += 1

ax[0][0].plot(range(count), r_avg_loss)
ax[0][0].set_title('Rolling avg loss per episode')

ax[0][1].plot(range(count), r_avg_rewards)
ax[0][1].set_title('Rolling avg reward per episode')

ax[1][0].plot(range(count), r_avg_profits)
ax[1][0].set_title('Rolling avg profit per episode')

ax[1][1].plot(range(count), r_avg_units_held)
ax[1][1].set_title('Rolling avg units held per episode')

ax[2][0].plot(range(count), r_avg_net_worth)
ax[2][0].set_title('Rolling avg net worth per episode')

ax[2][1].plot(range(count), r_avg_bal)
ax[2][1].set_title('Rolling avg balance per episode')

### Evaluation

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(16, 12))
r_avg_rewards = []
r_avg_profits = []
r_avg_bal = []
r_avg_units_held = []
r_avg_loss = []
r_avg_net_worth = []
count = 0

for _, log in a2c.eval_logs.items():
    r_avg_rewards.append(log['r_avg_reward'])
    r_avg_profits.append(log['r_avg_profit'])
    r_avg_bal.append(log['r_avg_bal'])
    r_avg_units_held.append(log['r_avg_units_held'])
    r_avg_loss.append(log['r_avg_loss'])
    r_avg_net_worth.append(log['r_avg_net_worth'])
    count += 1

ax[0][0].plot(range(count), r_avg_loss)
ax[0][0].set_title('Rolling avg loss per episode')

ax[0][1].plot(range(count), r_avg_rewards)
ax[0][1].set_title('Rolling avg reward per episode')

ax[1][0].plot(range(count), r_avg_profits)
ax[1][0].set_title('Rolling avg profit per episode')

ax[1][1].plot(range(count), r_avg_units_held)
ax[1][1].set_title('Rolling avg units held per episode')

ax[2][0].plot(range(count), r_avg_net_worth)
ax[2][0].set_title('Rolling avg net worth per episode')

ax[2][1].plot(range(count), r_avg_bal)
ax[2][1].set_title('Rolling avg balance per episode')

In [ ]:
# torch.save(a2c.actor.state_dict(), '../models/a2c_cl_an/actor')
# torch.save(a2c.critic.state_dict(), '../models/a2c_cl_an/critic')

In [ ]:
import pickle
from datetime import datetime

temp = dict(a2c.logs)
with open(f"../pickles/a2c_logs_{datetime.now()}.pickle","wb") as f:
    pickle.dump(temp, f, pickle.HIGHEST_PROTOCOL)

### Observations

- A2C performs well when actor is chained with LSTM and critic with Neural Network
- If the actor is chained with Neural Network we dont seem to learn the task